In [3]:
import numpy as np
import glob
import os
import imageio
from keras.models import Model
from keras.layers import *
from keras import backend as K
from keras.optimizers import Adam
from keras.datasets import cifar10
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255 - 0.5
x_test = x_test.astype('float32') / 255 - 0.5
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)
img_dim = x_train.shape[1]


z_dim = 256 # 隐变量维度
alpha = 0.5 # 全局互信息的loss比重
beta = 1.5 # 局部互信息的loss比重
gamma = 0.01 # 先验分布的loss比重


# 编码器（卷积与最大池化）
x_in = Input(shape=(img_dim, img_dim, 3))
x = x_in

for i in range(3):
    x = Conv2D(z_dim // 2**(2-i),
               kernel_size=(3,3),
               padding='SAME')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    x = MaxPooling2D((2, 2))(x)

feature_map = x # 截断到这里，认为到这里是feature_map（局部特征）
feature_map_encoder = Model(x_in, x)


for i in range(2):
    x = Conv2D(z_dim,
               kernel_size=(3,3),
               padding='SAME')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

x = GlobalMaxPooling2D()(x) # 全局特征

z_mean = Dense(z_dim)(x) # 均值，也就是最终输出的编码
z_log_var = Dense(z_dim)(x) # 方差，这里都是模仿VAE的


encoder = Model(x_in, z_mean) # 总的编码器就是输出z_mean


# 重参数技巧
def sampling(args):
    z_mean, z_log_var = args
    u = K.random_normal(shape=K.shape(z_mean))
    return z_mean + K.exp(z_log_var / 2) * u


# 重参数层，相当于给输入加入噪声
z_samples = Lambda(sampling)([z_mean, z_log_var])
prior_kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var))


# shuffle层，打乱第一个轴
def shuffling(x):
    idxs = K.arange(0, K.shape(x)[0])
    idxs = K.tf.random_shuffle(idxs)
    return K.gather(x, idxs)


# 与随机采样的特征拼接（全局）
z_shuffle = Lambda(shuffling)(z_samples)
z_z_1 = Concatenate()([z_samples, z_samples])
z_z_2 = Concatenate()([z_samples, z_shuffle])

# 与随机采样的特征拼接（局部）
feature_map_shuffle = Lambda(shuffling)(feature_map)
z_samples_repeat = RepeatVector(4 * 4)(z_samples)
z_samples_map = Reshape((4, 4, z_dim))(z_samples_repeat)
z_f_1 = Concatenate()([z_samples_map, feature_map])
z_f_2 = Concatenate()([z_samples_map, feature_map_shuffle])


# 全局判别器
z_in = Input(shape=(z_dim*2,))
z = z_in
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(1, activation='sigmoid')(z)

GlobalDiscriminator = Model(z_in, z)

z_z_1_scores = GlobalDiscriminator(z_z_1)
z_z_2_scores = GlobalDiscriminator(z_z_2)
global_info_loss = - K.mean(K.log(z_z_1_scores + 1e-6) + K.log(1 - z_z_2_scores + 1e-6))


# 局部判别器
z_in = Input(shape=(None, None, z_dim*2))
z = z_in
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(1, activation='sigmoid')(z)

LocalDiscriminator = Model(z_in, z)

z_f_1_scores = LocalDiscriminator(z_f_1)
z_f_2_scores = LocalDiscriminator(z_f_2)
local_info_loss = - K.mean(K.log(z_f_1_scores + 1e-6) + K.log(1 - z_f_2_scores + 1e-6))

# 用来训练的模型
model_train = Model(x_in, [z_z_1_scores, z_z_2_scores, z_f_1_scores, z_f_2_scores])
model_train.add_loss(alpha * global_info_loss + beta * local_info_loss + gamma * prior_kl_loss)
model_train.compile(optimizer=Adam(1e-3))

model_train.fit(x_train, epochs=50, batch_size=64)
model_train.save_weights('infomax.cifar10.weights')


# 输出编码器的特征
zs = encoder.predict(x_train, verbose=True)

/share/nanlh/work/pprli/pprli_env/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
50000/50000 [==============================] - 16s 327us/step - loss: 0.8016
Epoch 2/50
50000/50000 [==============================] - 14s 273us/step - loss: 0.4242
Epoch 3/50
50000/50000 [==============================] - 14s 274us/step - loss: 0.3494
Epoch 4/50
50000/50000 [==============================] - 14s 273us/step - loss: 0.3232
Epoch 5/50
50000/50000 [==============================] - 14s 273us/step - loss: 0.3058
Epoch 6/50
50000/50000 [==============================] - 14s 274us/step - loss: 0.2964
Epoch 7/50
50000/50000 [==============================] - 14s 272us/step - loss: 0.2807
Epoch 8/50
50000/50000 [==============================] - 14s 274us/step - loss: 0.2748
Epoch 9/50
50000/50000 [==============================] - 14s 274us/step - loss: 0.2684
Epoch 10/50
50000/50000 [==============================] - 14s 274us/step - loss: 0.2553
Epoch 11/50
50000/50000 [==============================] - 14s 274us/step - loss: 0.2616
Epoch 12/50
50000/50000 [=====

In [ ]:
def get_ae():
    x_in = Input(shape=(z_dim,))
    x = x_in
    return encoder_ae

In [4]:
from keras.utils import to_categorical
num_classes = 10
x_in = Input(shape=(z_dim,))
x = x_in
x = Dense(z_dim//2, activation='relu')(x)
x = Dense(num_classes, activation='softmax')(x)
classifier = Model(x_in, x)
classifier.compile(loss='categorical_crossentropy', optimizer=Adam(1e-3), metrics=['acc'])
y_train_use = to_categorical(y_train, num_classes=num_classes)
y_test_use = to_categorical(y_test, num_classes=num_classes)
x_train_use = encoder.predict(x_train)
x_test_use = encoder.predict(x_test)
classifier.fit(x_train_use, y_train_use, validation_data=(x_test_use, y_test_use), epochs=50, batch_size=64)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 2s 50us/step - loss: 1.4995 - acc: 0.4636 - val_loss: 1.4157 - val_acc: 0.4890
Epoch 2/50
50000/50000 [==============================] - 2s 44us/step - loss: 1.3681 - acc: 0.5120 - val_loss: 1.3633 - val_acc: 0.5137
Epoch 3/50
50000/50000 [==============================] - 2s 45us/step - loss: 1.3325 - acc: 0.5228 - val_loss: 1.3398 - val_acc: 0.5214
Epoch 4/50
50000/50000 [==============================] - 2s 43us/step - loss: 1.3094 - acc: 0.5355 - val_loss: 1.3388 - val_acc: 0.5170
Epoch 5/50
50000/50000 [==============================] - 2s 44us/step - loss: 1.2902 - acc: 0.5412 - val_loss: 1.3281 - val_acc: 0.5271
Epoch 6/50
50000/50000 [==============================] - 2s 43us/step - loss: 1.2790 - acc: 0.5442 - val_loss: 1.3271 - val_acc: 0.5295
Epoch 7/50
50000/50000 [==============================] - 2s 42us/step - loss: 1.2702 - acc: 0.5490 - val_loss: 1.3154 - val_acc